# Application NLP : Univers Musical de Charles Aznavour

In [1]:
# ======= Importation des bibliothèques =======
#!pip install spacy nltk textblob langdetect vaderSentiment deep-translator textblob_fr pandas numpy scikit-learn

# NLP (Traitement du langage naturel)
import spacy  
import nltk  
from nltk.corpus import stopwords  
from nltk.tokenize import word_tokenize  
from nltk.stem import SnowballStemmer  
from nltk.sentiment import SentimentIntensityAnalyzer  
from textblob import TextBlob
from langdetect import detect  
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer 
import subprocess
from deep_translator import GoogleTranslator
from textblob import TextBlob
from textblob_fr import PatternTagger, PatternAnalyzer
import langid

# Manipulation des données
import pandas as pd  
import numpy as np  
from sklearn.preprocessing import OneHotEncoder  

# Traitement de texte et Regex
import re  
import unicodedata  

# ======= Téléchargement des ressources NLTK si nécessaire =======
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('vader_lexicon')
nltk.download('punkt_tab')

[nltk_data] Downloading package stopwords to C:\Users\DELL
[nltk_data]     3420\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\DELL
[nltk_data]     3420\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package vader_lexicon to C:\Users\DELL
[nltk_data]     3420\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt_tab to C:\Users\DELL
[nltk_data]     3420\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

## Chargement de la base de donnée

In [12]:
# A changer
artist = "Charles Aznavour"
file_name = f"{artist.strip().replace(' ', '_').lower()}.csv"
base = pd.read_csv(file_name)
base = base[base['paroles'].notna()]

In [13]:
base.head()

,title,url,paroles
0,A Blue Like The Blue Of Your Eyes,https://paroles2chansons.lemonde.fr/paroles-ch...,A blue like the blue of your eyes When they're...
1,À chacun sa bohème,https://paroles2chansons.lemonde.fr/paroles-ch...,Je vous parle d'un temps La bohème Je vous par...
5,A Ma Fille,https://paroles2chansons.lemonde.fr/paroles-ch...,Je sais qu’un jour viendra Car la vie le comma...
6,À ma manière,https://paroles2chansons.lemonde.fr/paroles-ch...,Le jour Qui pour toujours Verra le lourd Ridea...
7,A mia foglia,https://paroles2chansons.lemonde.fr/paroles-ch...,"Il giorno arriverà che temo, ma che attendo, I..."


In [14]:
base.shape

(693, 3)

# Chargement des modèles spaCy pour différentes langues

In [15]:
try:
    nlp_fr = spacy.load('fr_core_news_sm')
    nlp_en = spacy.load("en_core_web_sm")
    print("Les modèles sont déjà installés.")
except OSError:
    print("Les modèles ne sont pas installés. Installation en cours...")
    subprocess.run(["python", "-m", "spacy", "download", "en_core_web_sm"])
    subprocess.run(["python", "-m", "spacy", "download", "fr_core_web_sm"])
    nlp_fr = spacy.load('fr_core_news_sm')
    nlp_en = spacy.load("en_core_web_sm")
    print("Installations terminées et modèles chargés avec succès.")

Les modèles sont déjà installés.


# Détecter la langue principale d'un texte.

In [16]:
def detect_and_translate(text):
    language_mapping = {
        'en': 'english',
        'fr': 'french',
        'es': 'spanish',
        'it': 'italian',
        'pt': 'portuguese',
        'de': 'german',
        'nl': 'dutch',
        'ru': 'russian'
    }
    try:
        detected_code, _ = langid.classify(text)  # Utilisation de langid
        detected_language = language_mapping.get(detected_code, detected_code)

        if detected_code == "fr":
            return detected_language, text  # Pas de traduction

        translated_lyrics = GoogleTranslator(source=detected_code, target='fr').translate(text)
        return detected_language, translated_lyrics
    except Exception as e:
        return 'unknown', text

# Prétraitement des paroles

In [17]:
def preprocess_text(text):
    """
    Prétraite le texte : suppression des caractères spéciaux, tokenisation,
    suppression des stop words, stemming/lemmatisation et normalisation
    """

    # Conversion en minuscules
    text = text.lower()

    # Normalisation Unicode
    text = unicodedata.normalize('NFKD', text).encode('ASCII', 'ignore').decode('utf-8')

    # Suppression des caractères spéciaux, des chiffres et des sauts de ligne
    text = re.sub(r'[^a-zA-Z\s]', ' ', text)
    text = re.sub(r'><', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()

    # Tokenisation
    words = word_tokenize(text)

    # Utilisation du paramètre language pour les stop words
    stop_words = set(stopwords.words('french'))
    words = [word for word in words if word not in stop_words]

    # Stemming avec la langue spécifiée
    stemmer = SnowballStemmer('french')
    words = [stemmer.stem(word) for word in words]

    return ' '.join(words)

# Analyse les sentiments d'une chanson complète

In [18]:
def analyze_sentiment(text):
    """
    Analyse le sentiment d'un texte en français.
    Retourne un score entre -1 (très négatif) et 1 (très positif).
    
    Args:
        text (str): Texte en français à analyser
        
    Returns:
        float: Score de sentiment entre -1 et 1
    """
    try:
        # Création d'un TextBlob avec le tagger et analyzer français
        blob = TextBlob(text, pos_tagger=PatternTagger(), analyzer=PatternAnalyzer())
        
        # Récupération du score de polarité
        polarity = blob.sentiment[0]
        
        # Normalisation du score entre -1 et 1
        return float(polarity)
        
    except Exception as e:
        print(f"Erreur lors de l'analyse du sentiment: {e}")
        return 0.0

In [19]:
def analyze_song_lyrics(title, lyrics):
    """
    Analyse les sentiments d'une chanson complète
    """
    # Nettoyage initial des lyrics (suppression des crochets et guillemets)
    lyrics = re.sub(r'[\[\]"]', '', lyrics)
    
    # Détection de la langue et traduction en francais
    language, translated_lyrics = detect_and_translate(lyrics)
    
    # Prétraitement
    cleaned_text = preprocess_text(translated_lyrics)
    
    # Analyse des sentiments
    sentiment_score = analyze_sentiment(cleaned_text)
    
    # Classification du sentiment en fonction du score
    if sentiment_score > 0.1:
        sentiment = "Positif"
    elif sentiment_score < -0.1:
        sentiment = "Négatif"
    else:
        sentiment = "Neutre"
    
    return {
        'titre': title,
        'langue': language,
        'score': sentiment_score,
        'sentiment': sentiment,
        'parole': cleaned_text
    }

# Analyse de chaque chanson

In [20]:
results = []
for song in base.to_dict(orient='records'):
    result = analyze_song_lyrics(song['title'], song['paroles'])
    results.append(result)

# Création d'un DataFrame pour afficher les résultats

In [21]:
df_results = pd.DataFrame(results)
df_results = df_results[df_results['langue'] != "unknown"]

In [22]:
file_name = f"{artist.strip().replace(' ', '_')}_sentiments.xlsx"
df_results.to_excel(file_name, index=False)

In [ ]:
import subprocess

subprocess.run(["streamlit", "run", r"3_app.py"])